In [4]:
import pandas as pd
import numpy as np
import os
import gc
import copy
import json
import sys
from pathlib import Path
from datetime import datetime, timedelta, date
import time
from dateutil.relativedelta import relativedelta 

import pyarrow.parquet as pq
import pyarrow as pa

from tqdm import tqdm

pd.options.display.max_rows = 100
pd.options.display.max_columns = 100

pd.options.display.float_format = '{:,.2f}'.format

import warnings
warnings.filterwarnings("ignore")

#import pytorch_lightning as pl
#random_seed=1
#pl.seed_everything(random_seed)

In [ ]:
from datasets import load_dataset

In [5]:
train_file = "tweets.jsonl"

In [1]:
!ls

classified_users.jsonl	seed_users.jsonl  tweets.jsonl
relationships.jsonl	tcc.ipynb


In [10]:
%%time
num_lines = sum(1 for line in open(train_file))
print(f'Num. of samples in train dataset: {num_lines:,}')

Num. of samples in train dataset: 179,530,343
CPU times: user 9min 17s, sys: 6min 29s, total: 15min 46s
Wall time: 36min 53s


In [8]:
for line in open(train_file):
    data = json.loads(line)
    print(json.dumps(data, indent=4))
    break

"{\"tweet\": {\"original_tweet\": {\"tweet\": {\"source\": \"<a href=\\\"http://twitter.com\\\" rel=\\\"nofollow\\\">Twitter Web Client</a>\", \"coordinates\": null, \"retweet\": false, \"text\": \"This is a heartbreakingly beautiful display of composure and grace https://t.co/67UWsec1BQ\", \"user\": {\"protected\": false, \"verified\": true, \"followers_count\": 4986951, \"favourites_count\": 110, \"created_at\": {\"$date\": 1243910139000}, \"name\": \"Seth Meyers\", \"profile_image_url\": \"https://pbs.twimg.com/profile_images/267298914/n700068668_5523.jpg\", \"id\": \"44039298\", \"description\": null, \"statuses_count\": 7802, \"lang\": \"en\", \"listed_count\": 20504, \"friends_count\": 635, \"username\": \"sethmeyers\", \"geo_enabled\": false}, \"created_at\": {\"$date\": 1502899689000}}, \"_id\": 897852491525742596}, \"source\": \"<a href=\\\"http://twitter.com/download/android\\\" rel=\\\"nofollow\\\">Twitter for Android</a>\", \"coordinates\": null, \"retweet\": true, \"text\"

In [1]:
with open(train_file, 'r') as file:
    for line in file:
        # Load the JSON from the line
        data = json.loads(line)

        # Pretty print the JSON
        print(json.dumps(data, indent=4))

        # Stop reading after the first line
        break


NameError: name 'train_file' is not defined

In [2]:
!ls ../../Downloads

archive.zip			  placaMoto.jpeg
balenaEtcher-1.18.4-x64.AppImage  president_polls_historical.csv
PericiaImagem.docx		  Win11_22H2_BrazilianPortuguese_x64v2.iso
placaMoto2.jpeg


In [3]:
import pandas as pd
df20 = pd.read_csv('../../Downloads/president_polls_historical.csv')
df16 = pd.read_csv('../../Downloads/presidential_polls.csv')

/tmp/ipykernel_3266/3659896690.py:2: DtypeWarning: Columns (14,15,21,23) have mixed types. Specify dtype option on import or set low_memory=False.
  df20 = pd.read_csv('../../Downloads/president_polls_historical.csv')


In [6]:
df20.drop_duplicates().reset_index()


,index,poll_id,pollster_id,pollster,sponsor_ids,sponsors,display_name,pollster_rating_id,pollster_rating_name,fte_grade,...,election_date,stage,nationwide_batch,ranked_choice_reallocated,ranked_choice_round,party,answer,candidate_id,candidate_name,pct
0,0,73830,940,Lake Research,1587,National Women's Law Center,Lake Research Partners,165,Lake Research Partners,A/B,...,11/3/20,general,False,False,NaN,DEM,Biden,13256,Joe Biden,51.0
1,1,73830,940,Lake Research,1587,National Women's Law Center,Lake Research Partners,165,Lake Research Partners,A/B,...,11/3/20,general,False,False,NaN,REP,Trump,13254,Donald Trump,48.0
2,2,74946,1677,The Political Matrix/The Listener Group,NaN,NaN,The Political Matrix/The Listener Group,677,The Political Matrix/The Listener Group,C/D,...,11/3/20,general,False,False,NaN,DEM,Biden,13256,Joe Biden,44.2
3,3,74946,1677,The Political Matrix/The Listener Group,NaN,NaN,The Political Matrix/The Listener Group,677,The Political Matrix/The Listener Group,C/D,...,11/3/20,general,False,False,NaN,REP,Trump,13254,Donald Trump,48.0
4,4,72621,383,PPP,NaN,NaN,Public Policy Polling,263,Public Policy Polling,A-,...,11/3/20,general,False,False,NaN,DEM,Biden,13256,Joe Biden,49.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16699,16699,57026,383,PPP,1363,DEM (partisan),Public Policy Polling,263,Public Policy Polling,A-,...,11/3/20,general,False,False,NaN,REP,Trump,13254,Donald Trump,46.0
16700,16700,57026,383,PPP,1363,DEM (partisan),Public Policy Polling,263,Public Policy Polling,A-,...,11/3/20,general,False,False,NaN,DEM,Warren,13258,Elizabeth Warren,43.0
16701,16701,57026,383,PPP,1363,DEM (partisan),Public Policy Polling,263,Public Policy Polling,A-,...,11/3/20,general,False,False,NaN,REP,Trump,13254,Donald Trump,49.0
16702,16702,57025,399,Rasmussen (Pulse Opinion Research),NaN,NaN,Rasmussen Reports,277,Rasmussen Reports,B,...,11/3/20,general,False,False,NaN,DEM,Obama,13253,Michelle Obama,50.0


In [7]:
df16.drop_duplicates().reset_index()

,index,cycle,branch,type,matchup,forecastdate,state,startdate,enddate,pollster,...,adjpoll_clinton,adjpoll_trump,adjpoll_johnson,adjpoll_mcmullin,multiversions,url,poll_id,question_id,createddate,timestamp
0,0,2016,President,polls-plus,Clinton vs. Trump vs. Johnson,11/1/16,U.S.,10/25/2016,10/31/2016,Google Consumer Surveys,...,42.64140,40.86509,5.675099,NaN,NaN,https://datastudio.google.com/u/0/#/org//repor...,47940,74999,11/1/16,15:09:38 1 Nov 2016
1,1,2016,President,polls-plus,Clinton vs. Trump vs. Johnson,11/1/16,U.S.,10/27/2016,10/30/2016,ABC News/Washington Post,...,43.29659,44.72984,3.401513,NaN,NaN,http://www.langerresearch.com/wp-content/uploa...,47881,74936,11/1/16,15:09:38 1 Nov 2016
2,2,2016,President,polls-plus,Clinton vs. Trump vs. Johnson,11/1/16,Virginia,10/27/2016,10/30/2016,ABC News/Washington Post,...,46.29779,40.72604,6.401513,NaN,NaN,https://www.washingtonpost.com/local/virginia-...,47880,74934,11/1/16,15:09:38 1 Nov 2016
3,3,2016,President,polls-plus,Clinton vs. Trump vs. Johnson,11/1/16,Florida,10/20/2016,10/24/2016,SurveyUSA,...,46.35931,45.30585,1.777730,NaN,NaN,http://www.baynews9.com/content/news/baynews9/...,47465,74252,10/25/16,15:09:38 1 Nov 2016
4,4,2016,President,polls-plus,Clinton vs. Trump vs. Johnson,11/1/16,U.S.,10/20/2016,10/25/2016,Pew Research Center,...,45.32744,42.20888,3.618320,NaN,NaN,http://www.people-press.org/2016/10/27/as-elec...,47616,74519,10/27/16,15:09:38 1 Nov 2016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10231,10231,2016,President,polls-only,Clinton vs. Trump vs. Johnson,11/1/16,Alabama,9/30/2016,10/13/2016,Ipsos,...,37.30964,54.76821,NaN,NaN,NaN,http://reuters.com/statesofthenation/,46817,73263,10/15/16,14:57:58 1 Nov 2016
10232,10232,2016,President,polls-only,Clinton vs. Trump vs. Johnson,11/1/16,Virginia,9/30/2016,10/6/2016,Ipsos,...,49.13094,39.41588,NaN,NaN,NaN,http://www.reuters.com/statesofthenation/,46675,72969,10/10/16,14:57:58 1 Nov 2016
10233,10233,2016,President,polls-only,Clinton vs. Trump vs. Johnson,11/1/16,Virginia,9/16/2016,9/22/2016,Ipsos,...,45.97130,39.97518,NaN,NaN,NaN,http://www.reuters.com/statesofthenation/,46096,72088,9/26/16,14:57:58 1 Nov 2016
10234,10234,2016,President,polls-only,Clinton vs. Trump vs. Johnson,11/1/16,North Carolina,6/20/2016,6/21/2016,Public Policy Polling,...,45.29390,46.66175,1.596946,NaN,NaN,http://www.publicpolicypolling.com/pdf/2015/PP...,44400,67363,6/23/16,14:57:58 1 Nov 2016
